In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import networkx as nx
import tqdm
from functools import cached_property
from math import lcm, gcd

from dataclasses import dataclass, field

Sequence:

- each number appears once;
- $x_n$ is relatively prime to $x_{n-1}$;
- $x_n$ is NOT relatively prime to $x_{n-2}$;
- $x_0 = 1$.
- $x_1 = 2$.
- $x_3 = 3$.

In [ ]:
from typing import Set, List, TypeVar, Callable, Generic, Iterable, Protocol, runtime_checkable, TypedDict, Optional

In [ ]:
x = {1,2,3}

In [ ]:
t = TypeVar("t")

Equivalence = Callable[[t, t], bool]
SortKey = Callable[[t], float]

relatively_prime: Equivalence[int] = lambda x, y: gcd(x, y) == 1
share_common_factor: Equivalence[int] = lambda x, y: gcd(x, y) != 1

In [ ]:
a = TypeVar("a")


@dataclass
class YellowstoneListSpec(Generic[a]):
    equivalence: Optional[Equivalence[a]] = None
    sort_key: Optional[SortKey[a]] = float
    items: Optional[List[a]] = None
        
    def on(self, items: List[a]):
        self.items = items
        return self
    
    def with_equivalence(self, equivalence: Equivalence[a]):
        self.equivalence = equivalence
        return self
        
    def with_sort_key(self, sort_key: SortKey[a]):
        self.sort_key = sort_key
        return self
    
    def ready(self):
        return YellowstoneList(self.items, self.equivalence, self.sort_key)

In [ ]:
class YellowstoneList(List[a]):
    
    def __init__(self, items: Iterable[a], equivalence: Equivalence[a], sort_key: SortKey[a]):
        super().__init__(items)
        self.equivalence = equivalence
        self.sort_key = sort_key
        self.queued = set(super().__iter__())
    
    @staticmethod
    def on(items: List[a]) -> YellowstoneListSpec[a]:
        return YellowstoneListSpec().on(items)
    
    def permutation(self):
        return YellowstonePermutation(self)
    
    def __iter__(self):
        iterator = super().__iter__()
        yield from (x for x in iterator if x in self.queued)

    def iter_valid_items(self, prototype: a, antitype: a):
        valid_items = (
            x for x in self 
            if self.equivalence(x, prototype) 
            and not self.equivalence(x, antitype)
        )
        yield from sorted(valid_items, key=self.sort_key)
        
    def dequeue(self, x: a):
        try:
            self.queued.remove(x)
        except KeyError:
            raise ValueError("Item not currently queued")

    def __repr__(self):
        return "<" + ", ".join([str(x) for x in self]) + f">"

    
@dataclass
class YellowstonePermutation(Generic[a]):
    items: YellowstoneList[a]
    
    def step(self, last, current) -> Optional[a]:
        candidates = list(self.items.iter_valid_items(last, current))
        if len(candidates) == 0:
            return None
        else:
            return candidates[0]
    
    def run(self) -> List[a]:
        last = self.items.pop(0)
        current = self.items.pop(0)
        output = [last, current]
        while True:
            next_item = self.step(last, current)
            if next_item is None:
                break
            else:
                output.append(next_item)
                last = current
                current = next_item
                self.items.dequeue(next_item)
        return output
                

In [ ]:
x = YellowstoneList.on(range(2, 100)).with_equivalence(share_common_factor).ready()

In [ ]:
print(x.permutation().run())

In [ ]:
xs = filter(lambda x: x % 2 == 0, range(10))

In [4]:
from itertools import chain
from math import lcm, gcd

In [62]:
def yellowstone(items):
    x1 = 2
    x2 = 3
    reservoir = []
    loop = True
    while loop:
        candidate = None
        for candidate in chain(reservoir, items):
            if (gcd(candidate, x1) != 1) and (gcd(candidate, x2) == 1):
                break
            else:
                reservoir.append(candidate)
        else:
            loop = False
        x1 = x2
        x2 = candidate
        yield candidate
        reservoir = [x for x in sorted(set(reservoir)) if x != candidate]

SyntaxError: 'break' outside loop (1804056754.py, line 14)

In [ ]:
for k in yellowstone(iter([4, 5, 6, 7, 8, 9])):
    print(k)

4
9
8
